In [35]:
import numpy as np
from scipy.stats import multivariate_normal, norm
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

import diff_gaussian_rasterization
import diff_gaussian_rasterization_batched as dgrb
import torch
import pyquaternion
import os
import math
import glm
import matplotlib.pyplot as plt

In [36]:
@interact(
        sx=widgets.FloatSlider(min=0.1, max=10, value=1),
        sy=widgets.FloatSlider(min=0.1, max=10, value=1),
        r=widgets.FloatSlider(min=0, max=2*np.pi, value=0),
        cutoff=widgets.FloatSlider(min=0.0, max=1, value=0.1)
)
def g(sx, sy, r, cutoff):
    x = np.linspace(-10,10,100)
    y = np.linspace(-10,10,100)
    X, Y = np.meshgrid(x,y)

    S = np.array([[sx, 0], [0, sy]])
    R = np.array([[np.cos(r), -np.sin(r)], [np.sin(r), np.cos(r)]])
    Sigma = R @ S @ S.T @ R.T

    for dim in range(2):
        scale = np.sqrt(Sigma[dim, dim])
        line = norm.ppf(cutoff, loc=0, scale=scale)
        if dim == 0:
            plt.vlines([line], -10, 10, colors='r')
        elif dim == 1:
            plt.hlines([line], -10, 10, colors='r')


    Z = multivariate_normal.pdf(np.dstack((X, Y)), mean=[0, 0], cov=Sigma)
    plt.contourf(X, Y, Z)
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='sx', max=10.0, min=0.1), FloatSlider(value=1.0, desc…

In [37]:
device = torch.device("cuda")

# generate a random point cloud
C = 3
B = 1
N = 100
means3D = torch.cat(
    [
        torch.rand(B, N, 1, dtype=torch.float32, device=device) * 2 - 1,
        torch.rand(B, N, 1, dtype=torch.float32, device=device) * 2 - 1,
        torch.rand(B, N, 1, dtype=torch.float32, device=device) * 2 - 1,
    ],
    dim=2,
)
means2D = torch.zeros(B, N, 2, dtype=torch.float32, device=device)
opacities = torch.ones(B, N, 1, dtype=torch.float32, device=device)
scales = torch.ones(B, N, 3, dtype=torch.float32, device=device) * 0.1
rotations = (
    torch.from_numpy(np.stack([pyquaternion.Quaternion().elements for _ in range(N)]))
    .to(torch.float32)
    .to(device)
    .expand(B, N, 4)
)
colors_precomp = torch.rand(B, N, C, dtype=torch.float32, device=device)

In [39]:
def get_eye(theta, phi, r):
    """Get the position of the camera.

    Args:
        theta (float): azimuth angle in radians
        phi (float): elevation angle in degrees
        r (float): distance of the camera from the origin

    Returns:
        torch.Tensor: position of the camera
    """
    s = [
            r * math.sin(phi) * math.cos(theta),
            r * math.sin(phi) * math.sin(theta),
            r * math.cos(phi),
    ]

    return [s[0], s[1], s[2]]



def toTensor(m: glm.mat4) -> torch.Tensor:
    return torch.tensor(m.to_list(), dtype=torch.float32, device=device)


@interact(
        fov=widgets.FloatSlider(min=1, max=179, value=60),
        azimuth=widgets.FloatSlider(min=-180, max=180, value=0),
        elevation=widgets.FloatSlider(min=-89, max=89, value=0),
        radius=widgets.FloatSlider(min=0.01, max=20, value=0),
)
def g(fov, azimuth, elevation, radius):
    FoVx = np.deg2rad(fov)
    FoVy = np.deg2rad(fov)

    eye = get_eye(np.deg2rad(elevation), np.deg2rad(azimuth), radius)

    print(eye)

    rasterizer = dgrb.GaussianRasterizer(
        dgrb.GaussianRasterizationSettings(
            image_channels=C,
            image_height=512,
            image_width=512,
            tanfovx=math.tan(FoVx * 0.5),
            tanfovy=math.tan(FoVy * 0.5),
            bg=torch.zeros(C, dtype=torch.float32, device=device),
            scale_modifier=1.0,
            projmatrix=toTensor(glm.perspective(FoVy, 1, 0.2, 100)),
            sh_degree=1,
            campos=torch.tensor([0, 0, 0], dtype=torch.float32, device=device),
            prefiltered=False,
            debug=True,
        )
    )

    rendered_image, radii = rasterizer(
        means3D=means3D,
        means2D=means2D,
        colors_precomp=colors_precomp,
        opacities=opacities,
        scales=scales,
        rotations=rotations,
        viewmatrixes=toTensor(
            glm.scale([1, 1, -1]) * glm.lookAt(eye, [0, 0, 0], [0, 1, 0])
        ).expand(B, 4, 4),
    )

    # display the rendered image
    image_batch = rendered_image.permute(0, 2, 3, 1).cpu().numpy()
    batches = image_batch.shape[0]
    fig, ax = plt.subplots(batches, 1, figsize=(10, 10), squeeze=False)
    for i in range(batches):
        ax[i, 0].imshow(image_batch[i])
        ax[i, 0].axis("off")

interactive(children=(FloatSlider(value=60.0, description='fov', max=179.0, min=1.0), FloatSlider(value=0.0, d…